インタラクティブな可視化を実装

In [1]:
from IPython.core.magic import register_cell_magic
@register_cell_magic
def skip(line, cell):
    # 実行されないようにセルの内容を無視する
    print(f"Skipping cell:\n{cell}")

In [2]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import joblib

from geolib import geohash

import geopandas as gpd
import matplotlib.pyplot as plt
#from shapely.geometry import Point
from shapely import Point,Polygon
import geopy
import geopy.distance
#import geohash2
from geodatasets import get_path,data
from ipywidgets import interact,Select,Dropdown
import folium
from folium.plugins import TimestampedGeoJson
import branca.colormap as cm

from statsmodels.tsa.seasonal import seasonal_decompose,STL
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf

from tqdm import tqdm
from IPython.display import display

import sys
sys.path.append('../')

from utils import (Multiple_Charts,
                   ProcessData,
                   trans_chara,
                   fill_missing_values,
                   fill_missing_with_mode,
                   find_nearest_station_main_table,
                   calculate_distance,
                   convert_kanji_to_fullwidth_2,
                   encode_geohash,
                   to_cat_col,
                   TreeExecution,
                   VotingModel,
                   submisson_datetime,
                   check_dataframe_elements
                   )

try:
    # 通常のスクリプト環境
    script_dir = os.path.dirname(os.path.abspath(__file__))
except NameError:
    # Jupyter NotebookやIPython環境
    script_dir = os.getcwd()
print(script_dir)

# ベースフォルダのパスを作成
base_dir = os.path.abspath(os.path.join(script_dir, r"../.."))
print(base_dir)  # 実際のフルパスが表示されます

TOTAL_PRICE_LOG="取引価格（総額）_log"
PRICE_PER_AREA_LOG="単位面積あたりの取引価格_log"

PROJECT_DIRECYORY=base_dir

DATA_INTERMEDIATE_RESULTS_CHECKPOINTS=os.path.join(base_dir,"data/intermediate_results/checkpoints")

#TARGET="取引価格（総額）_log"

c:\Users\yuto2\projects\Nishika\apartment_price_prediction_summer_2024\code\EDA
c:\Users\yuto2\projects\Nishika\apartment_price_prediction_summer_2024


コンペデータ可視化(下処理なし)

In [3]:
files = glob.glob(os.path.join(base_dir, "data/competition_data/train/train/*.csv"))
    #r"C:\Users\yuto2\OneDrive\ドキュメント\Pythonフォルダ\Nishika\中古マンション価格予測_2024夏の部\ダウンロード_データ\train\train\*.csv")
data_list = []
for file in files:
    data_list.append(pd.read_csv(file, index_col=0))
df = pd.concat(data_list)
df_copy=df.copy()
print("trainの内容")
check_dataframe_elements(df)

df_test=pd.read_csv(os.path.join(base_dir,"data/competition_data/test.csv"),index_col=0)
df_test_copy=df_test.copy()
print("testの内容")
check_dataframe_elements(df_test)


# カラム 'A' の欠損値の数を確認
tre_missing_values_count = df['最寄駅：名称'].isnull().sum()
tes_missing_values_count = df_test['最寄駅：名称'].isnull().sum()
print(f"学習データの欠損値の数: {tre_missing_values_count}")
print(f"テストデータの欠損値の数: {tes_missing_values_count}")

C:\Users\yuto2\AppData\Local\Temp\ipykernel_11096\826571061.py:5: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  data_list.append(pd.read_csv(file, index_col=0))
C:\Users\yuto2\AppData\Local\Temp\ipykernel_11096\826571061.py:5: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  data_list.append(pd.read_csv(file, index_col=0))


trainの内容
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1009271 entries, 1046933 to 47011639
Data columns (total 28 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   種類            1009271 non-null  object 
 1   価格情報区分        1009271 non-null  object 
 2   地域            0 non-null        float64
 3   市区町村コード       1009271 non-null  int64  
 4   都道府県名         1009271 non-null  object 
 5   市区町村名         1009271 non-null  object 
 6   地区名           1008610 non-null  object 
 7   最寄駅：名称        998620 non-null   object 
 8   最寄駅：距離（分）     964322 non-null   object 
 9   間取り           976570 non-null   object 
 10  面積（㎡）         1009271 non-null  object 
 11  土地の形状         0 non-null        float64
 12  間口            0 non-null        float64
 13  延床面積（㎡）       0 non-null        float64
 14  建築年           979931 non-null   object 
 15  建物の構造         977046 non-null   object 
 16  用途            734705 non-null   object 
 17  今後の利用目的    

In [4]:
df=pd.read_csv(os.path.join(DATA_INTERMEDIATE_RESULTS_CHECKPOINTS,"preprocess_main_table.csv"),index_col=0)
df=df[['dataset', 'ID', '価格情報区分', '都道府県名', '市区町村名', '地区名', '住所不一致_Flag', '住所',
       '最寄駅：名称', '最寄駅：距離（分）', '徒歩時間_10分単位', 'x_api', 'y_api', 'lon', 'lat',
       '間取り', '面積（㎡）', '建築年', '建物の構造', '用途', '今後の利用目的', '都市計画', '建ぺい率（％）',
       '容積率（％）', '取引時点', '改装', '取引の事情等',"取引価格（総額）_log", '単位面積あたりの取引価格_log', '建築西暦年', '築年数',
       'log_面積（㎡）', '購入までの築年数', '面積（㎡）-最寄駅：距離（分）', '面積（㎡）-築年数',
       '面積（㎡）容積率（％）_combi', '旧耐震フラグ', '最寄駅までの距離', 'geohash_2', 'geohash_3',
       'geohash_4', 'geohash_5', 'geohash_6', 'geohash_7', '県_市_count',
       '建築年_count', '県_市_geohash_2_駅数', '県_市_geohash_3_駅数', '県_市_geohash_4_駅数',
       '県_市_geohash_5_駅数', '県_市_geohash_6_駅数', '県_市_geohash_7_駅数',
       '県_市_geohash_2_駅徒歩時間平均', '県_市_geohash_2_建築年平均', '県_市_geohash_3_駅徒歩時間平均',
       '県_市_geohash_3_建築年平均', '県_市_geohash_4_駅徒歩時間平均', '県_市_geohash_4_建築年平均',
       '県_市_geohash_5_駅徒歩時間平均', '県_市_geohash_5_建築年平均', '県_市_geohash_6_駅徒歩時間平均',
       '県_市_geohash_6_建築年平均', '県_市_geohash_7_駅徒歩時間平均', '県_市_geohash_7_建築年平均']]


In [5]:
#train,testの結合データをdf_allとして保存
df_all=df.copy()

# データフレームを分割
df_train = df[df['dataset'] == 'train'].set_index('ID')
df_test = df[df['dataset'] == 'test'].set_index('ID')

# datasetカラムを削除
df_train = df_train.drop(columns=['dataset'])
df_test = df_test.drop(columns=['dataset'])

df_train,cat_cols=to_cat_col(df_train)

# インデックスを読み込み
train_index = np.load(os.path.join(DATA_INTERMEDIATE_RESULTS_CHECKPOINTS,"train_index_l1_cv.npy"))
test_index = np.load(os.path.join(DATA_INTERMEDIATE_RESULTS_CHECKPOINTS,'test_index_l1_cv.npy'))
print(train_index)
print(test_index)
# インデックスを設定して順序を再現
df_train = df_train.loc[train_index]
df_test = df_test.loc[test_index]

print("Train DataFrame:")
print(df_train)
print("\nTest DataFrame:")
print(df_test)

[ 1046933  1079053  1170588 ... 47003811 47010902 47011639]
[ 1000107  1000109  1000112 ... 47037850 47037855 47037856]
Train DataFrame:
             価格情報区分 都道府県名   市区町村名    地区名  住所不一致_Flag              住所 最寄駅：名称  \
ID                                                                            
1046933   不動産取引価格情報   北海道  札幌市白石区    南郷通           0    北海道札幌市白石区南郷通  南郷７丁目   
1079053   不動産取引価格情報   北海道   札幌市西区  発寒１２条           0   北海道札幌市西区発寒１２条   発寒中央   
1170588      成約価格情報   北海道     江別市   大麻北町           0      北海道江別市大麻北町   森林公園   
1001637      成約価格情報   北海道  札幌市中央区   北５条西           0   北海道札幌市中央区北５条西  西２８丁目   
1014790   不動産取引価格情報   北海道  札幌市中央区  宮の森２条           0  北海道札幌市中央区宮の森２条  西２８丁目   
...             ...   ...     ...    ...         ...             ...    ...   
47001427  不動産取引価格情報   沖縄県     那覇市  首里山川町           0     沖縄県那覇市首里山川町     儀保   
47035975     成約価格情報   沖縄県  中頭郡北谷町     美浜           0     沖縄県中頭郡北谷町美浜  てだこ浦西   
47003811  不動産取引価格情報   沖縄県     那覇市    古波蔵           0       沖縄県那覇市古波蔵     

In [6]:
#%%skip
#無難で完成度高い_v2

import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Polygon
from ipywidgets import interact, Select, Dropdown,HBox
from IPython.display import display

# ジオハッシュの境界ボックスを計算する関数を定義
def geohash_to_polygon(gh):
    if pd.isna(gh):
        return None
    bounds = geohash.bounds(gh)
    return Polygon([
        (bounds[0][1], bounds[0][0]),
        (bounds[0][1], bounds[1][0]),
        (bounds[1][1], bounds[1][0]),
        (bounds[1][1], bounds[0][0]),
        (bounds[0][1], bounds[0][0])
    ])

# データフレームを読み込み、必要な列をグループ化してジオハッシュの境界ボックスを計算
df = df_train.groupby(["取引時点", "最寄駅：距離（分）", "geohash_2", "geohash_3", "geohash_4", "geohash_5", "geohash_6", "geohash_7"])[f"{TOTAL_PRICE_LOG}"].mean().reset_index()
# カウント集約を追加
df_count = df_train.groupby(["取引時点", "最寄駅：距離（分）", "geohash_2", "geohash_3", "geohash_4", "geohash_5", "geohash_6", "geohash_7"]).size().reset_index(name='count')

# 各ジオハッシュレベルでのデータを個別にフィルタリング
def process_geohash_level(level):
    df_mean = df.groupby(["取引時点", level])[f"{TOTAL_PRICE_LOG}","最寄駅：距離（分）"].mean().reset_index()
    df_mean['geometry'] = [geohash_to_polygon(gh) for gh in tqdm(df_mean[level],desc=f"Calculating geohash polygons:{level}")]

    df_count_level = df_count.groupby(["取引時点", level])['count'].sum().reset_index()
    df_merged = pd.merge(df_mean, df_count_level, on=["取引時点", level])

    gdf = gpd.GeoDataFrame(df_merged, geometry='geometry')
    gdf.set_crs(epsg=4326, inplace=True)
    
    #display(gdf)
    return gdf

gdf_geohash_2 = process_geohash_level("geohash_2")
gdf_geohash_3 = process_geohash_level("geohash_3")
gdf_geohash_4 = process_geohash_level("geohash_4")
gdf_geohash_5 = process_geohash_level("geohash_5")
gdf_geohash_6 = process_geohash_level("geohash_6")
gdf_geohash_7 = process_geohash_level("geohash_7")

# 全てのジオハッシュレベルのデータフレームを連結
gdf_all = pd.concat([gdf_geohash_2, gdf_geohash_3, gdf_geohash_4, gdf_geohash_5, gdf_geohash_6, gdf_geohash_7])
unique_dates = gdf_all['取引時点'].unique()
unique_geohash_levels = ['geohash_2', 'geohash_3', 'geohash_4', 'geohash_5', 'geohash_6', 'geohash_7']
#unique_distances=sorted(gdf_all['最寄駅：距離（分）'].unique())

date_selector = Select(options=unique_dates, description='取引時点:')
geohash_selector = Dropdown(options=unique_geohash_levels, description='Geohash Level:')
#distance_selector = Select(options=unique_distances, description='徒歩時間:')
selectors=HBox([date_selector, geohash_selector])
# 横並びに配置
def create_selectors():
    return HBox([date_selector, geohash_selector])
#display(selectors)



# 地図を更新する関数を定義
def update_map(selected_date, selected_geohash):
    m = folium.Map(location=[35.0, 135.0], zoom_start=5)
    gdf_dict = {
        'geohash_2': gdf_geohash_2,
        'geohash_3': gdf_geohash_3,
        'geohash_4': gdf_geohash_4,
        'geohash_5': gdf_geohash_5,
        'geohash_6': gdf_geohash_6,
        'geohash_7': gdf_geohash_7
    }
    selected_gdf = gdf_dict[selected_geohash][(gdf_dict[selected_geohash]['取引時点']==selected_date)]

    min_value = selected_gdf[f"{TOTAL_PRICE_LOG}"].min()
    max_value = selected_gdf[f"{TOTAL_PRICE_LOG}"].max()
    bins = [min_value, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, max_value]#ここをbinsに変数として与える
    
    # カラーマップを定義
    #colormap = cm.linear.BuPu_09.scale(3.0, 9.0)

    # Choropleth layer for the average price
    folium.Choropleth(
        geo_data=selected_gdf, #ジオデータフレームを指定
        name='choropleth', #レイヤーの名前
        data=selected_gdf, #データを指定
        columns=[selected_geohash, f"{TOTAL_PRICE_LOG}"], #ジオハッシュと平均価格の列を指定
        key_on=f'feature.properties.{selected_geohash}', #ジオハッシュ列をキーとして使用
        fill_color='YlGnBu', #塗りつぶしの色パレットを'YlOrRd' に変更（黄色から赤のグラデーション)'YlGnBu'
        fill_opacity=0.7, #塗りつぶしの不透明度
        line_opacity=0.2, #線の不透明度
        legend_name='平均価格', #凡例の名前を指定
        #bins=[5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0]  # 固定のビンを設定
        #bins=[3.0, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0]  # 固定のビンを設定
        #bins=[3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0]  # 固定のビンを設定
        #bins=bins
    ).add_to(m)

    # Add a feature group for the counts
    folium.GeoJson(
        selected_gdf,
        name='Counts',
        style_function=lambda x: {
            'fillColor': '#ffaf00',
            'color': '#ffaf00',
            'weight': 1,
            'dashArray': '5, 5'
        },
        tooltip=folium.GeoJsonTooltip(
            fields=['count', f"{TOTAL_PRICE_LOG}", '最寄駅：距離（分）'],
            aliases=['Count:', 'Average Price:', 'Distance (min):'],
            localize=True
        )
    ).add_to(m)

    folium.LayerControl().add_to(m)

    m.save("map.html")
    display(m)

# インタラクティブに地図を更新する

#distance_selector=Select(options=sorted(gdf_all["最寄駅：距離（分）"].unique()),description="徒歩時間:")
interact(update_map, selected_date=date_selector, selected_geohash=geohash_selector)

C:\Users\yuto2\AppData\Local\Temp\ipykernel_11096\3618007817.py:31: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_mean = df.groupby(["取引時点", level])[f"{TOTAL_PRICE_LOG}","最寄駅：距離（分）"].mean().reset_index()
Calculating geohash polygons:geohash_2: 100%|██████████| 351/351 [00:00<00:00, 6676.98it/s]
C:\Users\yuto2\AppData\Local\Temp\ipykernel_11096\3618007817.py:31: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_mean = df.groupby(["取引時点", level])[f"{TOTAL_PRICE_LOG}","最寄駅：距離（分）"].mean().reset_index()
Calculating geohash polygons:geohash_3: 100%|██████████| 2279/2279 [00:00<00:00, 7234.42it/s]
C:\Users\yuto2\AppData\Local\Temp\ipykernel_11096\3618007817.py:31: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_mean = df.groupby(["取引時点", level])[f"{TO

interactive(children=(Select(description='取引時点:', options=(2005.75, 2005.99, 2006.25, 2006.5, 2006.75, 2006.99…

<function __main__.update_map(selected_date, selected_geohash)>